# 06: Standard Library Packages

KindScript ships pre-built architectural patterns as packages. Instead of writing your own Kind definitions and contracts from scratch, you can import them:

| Package | Pattern | Layers | Contracts |
|---------|---------|--------|-----------|
| `@kindscript/clean-architecture` | Clean Architecture | domain, application, infrastructure | noDependency, purity |
| `@kindscript/hexagonal` | Hexagonal / Ports & Adapters | domain, ports, adapters | noDependency, mustImplement, purity |
| `@kindscript/onion` | Onion Architecture | core, domainServices, applicationServices, infrastructure | noDependency, purity |

This notebook shows what each package provides and how to use them.

## Setup

In [ ]:
const PROJECT_ROOT = Deno.cwd().replace(/\/notebooks$/, "");
const KSC = PROJECT_ROOT + "/dist/infrastructure/cli/main.js";

async function ksc(...args: string[]): Promise<{ code: number; output: string }> {
  const cmd = new Deno.Command("node", {
    args: [KSC, ...args],
    stdout: "piped",
    stderr: "piped",
  });
  const { code, stdout, stderr } = await cmd.output();
  const output = (new TextDecoder().decode(stdout) + new TextDecoder().decode(stderr)).trim();
  if (output) console.log(output);
  console.log(`\nExit code: ${code}`);
  return { code, output };
}

console.log("Setup complete.");

---

## Package 1: `@kindscript/clean-architecture`

Clean Architecture (Robert C. Martin) separates code into three concentric layers:
- **Domain** — pure business logic, no external dependencies
- **Application** — use case orchestration
- **Infrastructure** — database adapters, HTTP clients, etc.

### What the package provides

In [ ]:
// Let's look at what the package defines
const cleanArchSrc = Deno.readTextFileSync(`${PROJECT_ROOT}/packages/clean-architecture/index.ts`);
console.log(cleanArchSrc);

The package exports:
- **`CleanContext`** — a Kind with `domain`, `application`, and `infrastructure` children
- **`DomainLayer`**, **`ApplicationLayer`**, **`InfrastructureLayer`** — leaf kinds for each layer
- **`cleanArchitectureContracts`** — pre-configured contracts:
  - `noDependency`: domain cannot import from application or infrastructure; application cannot import from infrastructure
  - `purity`: domain layer must have no I/O imports

### Using the package

With the stdlib installed, your `architecture.ts` becomes minimal — just the instance mapping:

In [ ]:
// Create a project using the stdlib package
const DEMO = Deno.makeTempDirSync({ prefix: "ksc-stdlib-" });
Deno.mkdirSync(`${DEMO}/src/domain`, { recursive: true });
Deno.mkdirSync(`${DEMO}/src/application`, { recursive: true });
Deno.mkdirSync(`${DEMO}/src/infrastructure`, { recursive: true });

// Install the package (copy from our packages/ directory)
Deno.mkdirSync(`${DEMO}/node_modules/@kindscript/clean-architecture`, { recursive: true });
Deno.copyFileSync(
  `${PROJECT_ROOT}/packages/clean-architecture/index.ts`,
  `${DEMO}/node_modules/@kindscript/clean-architecture/index.ts`,
);

// Source files
Deno.writeTextFileSync(`${DEMO}/src/domain/entity.ts`, `
export interface Order { id: string; total: number; }
`.trimStart());

Deno.writeTextFileSync(`${DEMO}/src/application/handler.ts`, `
import { Order } from '../domain/entity';
export function process(o: Order): void { console.log(o.id); }
`.trimStart());

Deno.writeTextFileSync(`${DEMO}/src/infrastructure/repository.ts`, `
import { Order } from '../domain/entity';
export function save(o: Order): void { /* persist */ }
`.trimStart());

Deno.writeTextFileSync(`${DEMO}/tsconfig.json`, JSON.stringify({
  compilerOptions: { target: "ES2020", module: "commonjs", strict: true, rootDir: ".", outDir: "dist" },
  include: ["src/**/*.ts", "architecture.ts"],
}, null, 2));

// The architecture.ts — just an instance mapping, types come from the package
Deno.writeTextFileSync(`${DEMO}/architecture.ts`, `
import { CleanContext } from '@kindscript/clean-architecture';

export const app: CleanContext = {
  kind: "CleanContext",
  location: "src",
  domain: {
    kind: "DomainLayer",
    location: "src/domain",
  },
  application: {
    kind: "ApplicationLayer",
    location: "src/application",
  },
  infrastructure: {
    kind: "InfrastructureLayer",
    location: "src/infrastructure",
  },
};
`.trimStart());

Deno.writeTextFileSync(`${DEMO}/kindscript.json`, JSON.stringify({
  definitions: ["architecture.ts"],
  packages: ["@kindscript/clean-architecture"],
}, null, 2));

console.log("=== architecture.ts (with stdlib) ===");
console.log(Deno.readTextFileSync(`${DEMO}/architecture.ts`));
console.log("=== kindscript.json ===");
console.log(Deno.readTextFileSync(`${DEMO}/kindscript.json`));

In [ ]:
// Check passes — contracts come from the package
console.log("=== ksc check with stdlib package ===");
await ksc("check", DEMO);

In [ ]:
// Introduce a violation — stdlib contracts still enforce
Deno.writeTextFileSync(`${DEMO}/src/domain/entity.ts`, `
import { save } from '../infrastructure/repository';

export interface Order { id: string; total: number; }
export function createAndSave(): void { save({ id: '1', total: 0 }); }
`.trimStart());

console.log("=== Violation with stdlib: domain imports from infrastructure ===");
await ksc("check", DEMO);

Deno.removeSync(DEMO, { recursive: true });

The stdlib contracts enforce the same rules — you get `KS70001: Forbidden dependency` just like with inline definitions.

### Inline vs. Stdlib comparison

| | Inline | Stdlib |
|---|--------|--------|
| `architecture.ts` size | ~50 lines (boilerplate + definitions + contracts) | ~15 lines (just the instance) |
| Contracts | You write them | Pre-configured by the package |
| Customization | Full control | Import and extend |
| Updates | Manual | Update the package version |

**Recommendation:** Start with the stdlib package, customize later if needed.

---

## Package 2: `@kindscript/hexagonal`

Hexagonal Architecture (Alistair Cockburn) — also called Ports & Adapters:
- **Domain** — core business logic
- **Ports** — interfaces defining how the domain talks to the outside world
- **Adapters** — implementations of ports for specific technologies

In [ ]:
const hexSrc = Deno.readTextFileSync(`${PROJECT_ROOT}/packages/hexagonal/index.ts`);
// Show just the exports (skip boilerplate)
const hexExports = hexSrc.split('\n').filter(l =>
  l.startsWith('export') || l.startsWith('/**') || l.startsWith(' *') || l.trim().startsWith('noDependency') || l.trim().startsWith('mustImplement') || l.trim().startsWith('purity')
).join('\n');
console.log(hexExports);

Hexagonal contracts are stricter than Clean Architecture:
- **`noDependency`**: domain cannot import from adapters
- **`mustImplement`**: every port interface must have an adapter class
- **`purity`**: domain must be free of I/O

The `mustImplement` contract is unique to hexagonal — it ensures you haven't defined a port without wiring up its adapter.

### Usage

```typescript
import { HexagonalContext } from '@kindscript/hexagonal';

export const app: HexagonalContext = {
  kind: "HexagonalContext",
  location: "src",
  domain: { kind: "DomainLayer", location: "src/domain" },
  ports: { kind: "PortsLayer", location: "src/ports" },
  adapters: { kind: "AdaptersLayer", location: "src/adapters" },
};
```

---

## Package 3: `@kindscript/onion`

Onion Architecture (Jeffrey Palermo) — concentric rings:
- **Core** — domain model, entities, value objects (innermost)
- **Domain Services** — logic spanning multiple entities
- **Application Services** — use case orchestration
- **Infrastructure** — external concerns (outermost)

In [ ]:
const onionSrc = Deno.readTextFileSync(`${PROJECT_ROOT}/packages/onion/index.ts`);
const onionExports = onionSrc.split('\n').filter(l =>
  l.startsWith('export') || l.startsWith('/**') || l.startsWith(' *') || l.trim().startsWith('noDependency') || l.trim().startsWith('purity')
).join('\n');
console.log(onionExports);

Onion Architecture has the most `noDependency` contracts — 5 rules enforcing that dependencies always point inward:
- Core cannot depend on anything
- Domain services cannot depend on infrastructure
- Application services cannot depend on infrastructure

### Usage

```typescript
import { OnionContext } from '@kindscript/onion';

export const app: OnionContext = {
  kind: "OnionContext",
  location: "src",
  core: { kind: "CoreLayer", location: "src/core" },
  domainServices: { kind: "DomainServicesLayer", location: "src/domain-services" },
  applicationServices: { kind: "ApplicationServicesLayer", location: "src/application-services" },
  infrastructure: { kind: "InfrastructureLayer", location: "src/infrastructure" },
};
```

---

## `ksc infer` with stdlib

When `ksc infer` detects that a `@kindscript/*` package is installed, it generates import-based definitions instead of inline stubs.

In [ ]:
// Create a project with the stdlib installed
const INFER_DEMO = Deno.makeTempDirSync({ prefix: "ksc-infer-stdlib-" });
Deno.mkdirSync(`${INFER_DEMO}/src/domain`, { recursive: true });
Deno.mkdirSync(`${INFER_DEMO}/src/application`, { recursive: true });
Deno.mkdirSync(`${INFER_DEMO}/src/infrastructure`, { recursive: true });
Deno.mkdirSync(`${INFER_DEMO}/node_modules/@kindscript/clean-architecture`, { recursive: true });

Deno.copyFileSync(
  `${PROJECT_ROOT}/packages/clean-architecture/index.ts`,
  `${INFER_DEMO}/node_modules/@kindscript/clean-architecture/index.ts`,
);

Deno.writeTextFileSync(`${INFER_DEMO}/src/domain/entity.ts`, `export interface Order { id: string; }\n`);
Deno.writeTextFileSync(`${INFER_DEMO}/src/application/handler.ts`, `import { Order } from '../domain/entity';\nexport function handle(o: Order) {}\n`);
Deno.writeTextFileSync(`${INFER_DEMO}/src/infrastructure/repository.ts`, `import { Order } from '../domain/entity';\nexport function save(o: Order) {}\n`);
Deno.writeTextFileSync(`${INFER_DEMO}/tsconfig.json`, JSON.stringify({
  compilerOptions: { target: "ES2020", module: "commonjs", strict: true, rootDir: ".", outDir: "dist" },
  include: ["src/**/*.ts"],
}, null, 2));

console.log("=== ksc infer with @kindscript/clean-architecture installed ===");
await ksc("infer", INFER_DEMO);

Deno.removeSync(INFER_DEMO, { recursive: true });

Notice the difference: instead of ~30 lines of boilerplate (Kind interface, ContractConfig, defineContracts), the generated `architecture.ts` starts with:

```typescript
import { CleanContext } from '@kindscript/clean-architecture';
```

Much cleaner. The contracts and type definitions come from the package.

---

## Summary

| Package | Best for | Key contract |
|---------|----------|-------------|
| `@kindscript/clean-architecture` | Most backend services, microservices | `noDependency` between all 3 layers |
| `@kindscript/hexagonal` | Systems with many external integrations | `mustImplement` (ports need adapters) |
| `@kindscript/onion` | Complex domain-heavy applications | 5 `noDependency` rules (concentric rings) |

All three include `purity` on the innermost layer — your domain logic stays free of I/O.